<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=cd284941aa5ab738500dc9a7677e2f196961cdfaef8d9597a05a8597e4c6ac37
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 11:15:57
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: 8.19 K (0.06%)
Current PnL: -25.18 L (-16.27%)
CY Booked + Current PnL: -10.89 L (-7.04%)
-------------------
Total profit:  1.65 L
Total loss:  -26.83 L
-------------------
Total Booked + Current PnL: 16.09 L (12.59%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.07%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 89.82 L (63.31%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,1.52,10.04,8.77,19.69,18504.0,19248.0,210990.0,-0.66,59.0,X-MC,2.50,78.0,1.04,1.49,17.52,XY25,NTT,AC
77,TTKPRESTIG,770.00,-0.04,-19.16,23.81,0.09,19398.0,-19308.0,81469.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,7.03,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.09,-1.19,23.52,22.05,22770.0,-1170.0,96810.0,-14.63,57.0,H-SC,6.65,164.0,-0.05,0.69,40.72,XR,ATH,FINANCE
43,ITC,452.00,0.06,-0.10,10.89,10.78,26144.0,-242.0,240076.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.13,X40,NTT,FMCG
52,NESTLEIND,1377.00,0.18,10.97,9.40,21.40,27686.0,29110.0,294536.0,2.90,56.0,X-LC,5.53,12.0,1.05,2.09,18.34,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.57,-1.86,34.93,32.42,87033.0,-4731.0,249165.0,-19.22,50.0,X-SC,10.68,85.0,-0.05,1.76,12.95,X40,ATH,FMCG
47,KANSAINER,340.00,2.73,-23.11,47.58,13.47,98653.0,-62325.0,207342.0,-68.31,51.0,H-SC,6.66,158.0,-0.63,1.47,5.43,XY24,NTT,PAINTS
63,SFL,1287.00,2.48,-41.13,115.09,26.63,177669.0,-107844.0,154374.0,16.95,49.0,M-SC,12.32,239.0,-0.61,1.09,5.69,XY24,NTT,MISC
74,TITAGARH,1548.00,2.37,-19.33,80.65,45.74,157568.0,-46804.0,195373.0,7.41,62.0,H-SC,6.41,174.0,-0.30,1.38,27.07,XY24,NTT,ENGINEERING
22,DIXON,18931.72,2.02,-12.83,44.62,26.07,75936.0,-25038.0,170183.0,-57.60,38.0,X-MC,12.37,56.0,-0.33,1.21,5.99,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.28,-18.42,84.82,50.79,97331.0,-25903.0,114750.0,4.75,69.0,H-SC,8.42,151.0,-0.27,0.81,33.61,XR,ATH,IT
83,WHIRLPOOL,2270.00,-1.20,-25.39,143.82,81.92,123185.0,-29146.0,85652.0,-56.34,31.0,M-SC,21.76,236.0,-0.24,0.61,2.17,XR,NTT,DURABLES
57,RAJOOENG,143.10,-1.19,-38.62,127.47,39.61,80191.0,-39590.0,62910.0,-58.53,31.0,H-SC,31.09,135.0,-0.49,0.45,0.00,AR,ATH,MISC
81,VBL,671.64,-1.10,-3.76,40.97,35.67,124539.0,-11867.0,303975.0,-15.14,53.0,X-LC,3.46,4.0,-0.10,2.15,9.58,X40N,ATH,FMCG
6,ASIANTILES,137.00,-1.09,-3.79,88.84,81.67,80180.0,-3558.0,90252.0,8127.78,68.0,L-SC,12.83,271.0,-0.04,0.64,74.78,XR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,1.45,-0.11,19.36,19.23,29097.0,-166.0,150294.0,16.96,77.0,L-LC,2.51,179.0,-0.01,1.06,21.14,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.57,-2.73,113.74,107.91,163349.0,-4028.0,143616.0,-20.96,52.0,M-SC,11.86,216.0,-0.02,1.02,7.08,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.18,-17.09,20.64,0.02,29870.0,-29839.0,144720.0,120.42,40.0,M-SC,11.38,234.0,-1.00,1.02,25.88,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.04,-19.16,23.81,0.09,19398.0,-19308.0,81469.0,89.11,38.0,M-SC,2.62,253.0,-1.00,0.58,7.03,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.77,-22.36,70.06,32.03,71709.0,-29482.0,102353.0,-16.04,59.0,H-SC,8.95,148.0,-0.41,0.72,14.64,OX40N,NTT,IT
47,KANSAINER,340.00,2.73,-23.11,47.58,13.47,98653.0,-62325.0,207342.0,-68.31,51.0,H-SC,6.66,158.0,-0.63,1.47,5.43,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,-1.0,10.21,56.31,72.27,105157.0,17298.0,186746.0,-5.63,68.0,M-LC,3.89,99.0,0.16,1.32,17.01,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.00,10.21,56.31,72.27,105157.0,17298.0,186746.0,-5.63,68.0,M-LC,3.89,99.0,0.16,1.32,17.01,XR,NTT,IT
25,FINCABLES,1641.55,-0.57,-2.73,113.74,107.91,163349.0,-4028.0,143616.0,-20.96,52.0,M-SC,11.86,216.0,-0.02,1.02,7.08,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.70,-3.57,114.39,106.72,136041.0,-4409.0,118927.0,-52.34,42.0,H-SC,4.49,139.0,-0.03,0.84,18.97,AR,ATH,MISC
6,ASIANTILES,137.00,-1.09,-3.79,88.84,81.67,80180.0,-3558.0,90252.0,8127.78,68.0,L-SC,12.83,271.0,-0.04,0.64,74.78,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.09,-1.19,23.52,22.05,22770.0,-1170.0,96810.0,-14.63,57.0,H-SC,6.65,164.0,-0.05,0.69,40.72,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.63,-18.41,90.55,55.47,179743.0,-44785.0,198501.0,-27.60,30.0,X-MC,15.62,64.0,-0.25,1.41,1.71,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.25,-18.29,75.41,43.32,100356.0,-29792.0,133080.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.18,X40N,ATH,ELECTRICAL
8,AWL,485.00,0.04,-24.22,102.55,53.50,234506.0,-73076.0,228675.0,-63.37,33.0,X-MC,5.88,58.0,-0.31,1.62,1.57,XY24,NTT,FMCG
3,ACC,3906.00,-0.58,-26.71,123.97,64.15,216204.0,-63551.0,174400.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
20,DABUR,735.00,-0.17,-3.75,49.30,43.70,117712.0,-9302.0,238766.0,-11.62,34.0,X-MC,2.12,72.0,-0.08,1.69,10.74,XY24,BTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.41,-2.68,24.47,21.14,54140.0,-6088.0,221251.0,-9.49,48.0,X-MC,1.03,74.0,-0.11,1.57,24.71,XY24,NTT,PAINTS
15,CAMS,950.0,1.43,1.04,23.20,24.48,59035.0,2607.0,254463.0,-80.65,53.0,X-SC,1.28,86.0,0.04,1.80,24.41,X40N,NTT,MISC
78,UNITDSPR,1644.0,-0.82,5.05,14.97,20.77,36820.0,11816.0,245960.0,-4.14,51.0,X-MC,1.97,62.0,0.32,1.74,12.31,X40N,NTT,BREWERIES
43,ITC,452.0,0.06,-0.10,10.89,10.78,26144.0,-242.0,240076.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.13,X40,NTT,FMCG
20,DABUR,735.0,-0.17,-3.75,49.30,43.70,117712.0,-9302.0,238766.0,-11.62,34.0,X-MC,2.12,72.0,-0.08,1.69,10.74,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.58,-26.71,123.97,64.15,216204.0,-63551.0,174400.0,-57.19,34.0,X-SC,4.20,82.0,-0.29,1.24,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.06,-20.08,77.06,41.51,162204.0,-52875.0,210490.0,-4.76,40.0,X-MC,7.86,61.0,-0.33,1.49,0.84,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.08,-38.74,120.71,35.20,95145.0,-49849.0,78821.0,3.39,36.0,X-SC,17.01,92.0,-0.52,0.56,1.10,X40,NTT,FOOTWEAR
8,AWL,485.00,0.04,-24.22,102.55,53.50,234506.0,-73076.0,228675.0,-63.37,33.0,X-MC,5.88,58.0,-0.31,1.62,1.57,XY24,NTT,FMCG
9,BAJAJHFL,181.50,-0.63,-18.41,90.55,55.47,179743.0,-44785.0,198501.0,-27.60,30.0,X-MC,15.62,64.0,-0.25,1.41,1.71,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.80,-27.68,97.50,42.84,45840.0,-17991.0,47015.0,-52.62,52.0,X-SC,33.80,83.0,-0.39,0.33,6.37,XY24,NTT,MISC
59,RELAXO,1176.00,1.08,-47.19,188.38,52.31,144423.0,-68494.0,76666.0,-43.76,48.0,X-SC,7.31,91.0,-0.47,0.54,2.62,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.08,-38.74,120.71,35.20,95145.0,-49849.0,78821.0,3.39,36.0,X-SC,17.01,92.0,-0.52,0.56,1.10,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-1.01,-1.81,26.36,24.08,32859.0,-2294.0,124656.0,-9.82,45.0,X-SC,6.45,89.0,-0.07,0.88,17.45,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.25,-18.29,75.41,43.32,100356.0,-29792.0,133080.0,-29.79,31.0,X-SC,5.98,90.0,-0.30,0.94,2.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.05,-8.94,32.56,20.71,112139.0,-33831.0,344406.0,-21.68,70.0,X-LC,3.21,1.0,-0.30,2.44,15.08,X40,ATH,IT
41,INFY,1972.00,-0.50,13.01,18.80,34.24,66473.0,40692.0,353580.0,-12.06,66.0,X-LC,6.15,2.0,0.61,2.50,22.36,X40,NTT,IT
75,TMPV,600.00,-1.06,-26.34,66.94,22.98,111389.0,-59496.0,166402.0,-23.08,50.0,X-LC,6.41,3.0,-0.53,1.18,4.66,XY24,NTT,AUTO
81,VBL,671.64,-1.10,-3.76,40.97,35.67,124539.0,-11867.0,303975.0,-15.14,53.0,X-LC,3.46,4.0,-0.10,2.15,9.58,X40N,ATH,FMCG
43,ITC,452.00,0.06,-0.10,10.89,10.78,26144.0,-242.0,240076.0,-40.34,58.0,X-LC,2.01,5.0,-0.01,1.70,5.13,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.60,-36.88,110.53,32.89,53867.0,-28472.0,48735.0,-701.92,57.0,L-MC,6.84,259.0,-0.53,0.35,34.21,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-0.21,-57.01,132.75,0.07,78293.0,-78199.0,58978.0,2061.62,70.0,L-SC,4.05,268.0,-1.00,0.42,46.68,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.09,-3.79,88.84,81.67,80180.0,-3558.0,90252.0,8127.78,68.0,L-SC,12.83,271.0,-0.04,0.64,74.78,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.09,-1.19,23.52,22.05,22770.0,-1170.0,96810.0,-14.63,57.0,H-SC,6.65,164.0,-0.05,0.69,40.72,XR,ATH,FINANCE
13,BSOFT,831.70,-1.28,-18.42,84.82,50.79,97331.0,-25903.0,114750.0,4.75,69.0,H-SC,8.42,151.0,-0.27,0.81,33.61,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.28,-18.42,84.82,50.79,97331.0,-25903.0,114750.0,4.75,69.0,H-SC,8.42,151.0,-0.27,0.81,33.61,XR,ATH,IT
56,RAJESHEXPO,518.00,-0.21,-57.01,132.75,0.07,78293.0,-78199.0,58978.0,2061.62,70.0,L-SC,4.05,268.0,-1.00,0.42,46.68,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,-1.09,-3.79,88.84,81.67,80180.0,-3558.0,90252.0,8127.78,68.0,L-SC,12.83,271.0,-0.04,0.64,74.78,XR,NTT,CERAMICS
61,ROUTE,2227.21,-0.79,-46.16,214.00,69.05,153307.0,-61427.0,71639.0,-56.49,53.0,H-SC,20.62,140.0,-0.40,0.51,10.48,SR,ATH,IT
17,COALINDIA,484.83,1.45,-0.11,19.36,19.23,29097.0,-166.0,150294.0,16.96,77.0,L-LC,2.51,179.0,-0.01,1.06,21.14,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,44.79
2,50,76.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.03
MC    30.74
LC    24.22
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.77
X40      23.03
X40N     12.93
XR        9.94
XY25      9.14
AR        8.90
OX40N     7.52
SR        1.00
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.21
H-SC    23.88
X-LC    20.73
M-SC    11.55
X-SC     8.05
H-MC     4.79
L-SC     1.55
M-MC     1.39
M-LC     1.32
H-LC     1.11
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-7.33,41.84
IT,13.12,-13.44,70.47
FINANCE,9.57,-17.98,69.29
MISC,7.29,-26.00,77.54
ELECTRICAL,5.96,-11.51,52.36
PAINTS,5.80,-12.64,29.37
INSURANCE,4.74,-1.28,35.94
PHARMA,4.04,-2.88,35.26
AUTO,2.85,-30.45,75.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140293.0,21
AR,1315701.0,10
XR,1260417.0,13
X40,969108.0,14
X40N,859218.0,9
OX40N,730318.0,10
XY25,349526.0,6
SR,279615.0,2
MH,77503.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3608828.0,25
M-SC,1426829.0,15
X-MC,1408478.0,16
X-LC,796921.0,11
X-SC,780895.0,8
H-MC,401708.0,3
L-SC,243587.0,3
M-LC,105157.0,1
H-LC,77304.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1255729.0      6
           AR         903218.0      5
M-SC       XY24       806372.0      6
H-SC       XR         761167.0      7
X-MC       X40        485128.0      7
           XY24       406358.0      3
           X40N       336284.0      4
H-SC       OX40N      331596.0      4
M-SC       OX40N      320429.0      5
X-SC       X40N       303814.0      3
X-LC       X40        301802.0      5
X-SC       XY24       294903.0      3
H-SC       SR         279615.0      2
X-LC       X40N       219120.0      2
H-MC       AR         210083.0      2
           XY24       191625.0      1
X-LC       XY24       185306.0      2
X-SC       X40        182178.0      2
X-MC       XY25       180708.0      2
M-SC       XR         174932.0      2
L-SC       XR         165294.0      2
M-SC       AR         125096.0      2
M-LC       XR         105157.0      1
X-LC       XY25        90693.0      2
L-SC       OX40N       78293.0      1
H-SC       MH          77503.0      1
H-LC       AR          77304.0      1
L-MC       XR          53867.0      1
M-MC       XY25        49028.0      1
L-LC       XY25        29097.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
